In [ ]:
#| default_exp formatting

In [ ]:
#| exporti 

from typing import (
    Union,
    Callable,
    TypeVar,
    Literal
)
from pretiumlake import BaseModel
from pydantic import root_validator
from pydantic.color import Color
import plotly.io as pio
from plotly import graph_objs as go
from typing import List
import math
from pandas import isnull
import pandas as pd
import datetime as dt

## String Formatting

### Format Factory

In [ ]:
#| exporti 

FormatterT = TypeVar("FormatterT",str,Callable)

In [ ]:
#| exporti

def format_factory(
    string_or_callable:FormatterT,
    null_format:str = '',
    **kwargs
)->Callable:
    """
    Returns a function factory to format a given value. 
    """
    if callable(string_or_callable):
        def _formatter(val,*args):

            if val=='' or isnull(val):
                return null_format

            
            return string_or_callable(val,*args,**kwargs)
        return _formatter
    
    def _formatter(val):
        if val=='' or isnull(val):
            return null_format

        return string_or_callable.format(val)
    return _formatter

In [ ]:
commas = format_factory("{:,.0f}")
assert commas(1234.0) == '1,234'

In [ ]:
def convert_snake_case(string:str,title:bool=False)->str:
    formatted = " ".join(x for x in string.split('_'))
    if title:
        return formatted.title()
    return formatted

snake_to_title = format_factory(convert_snake_case,title=True)
assert snake_to_title("snake_case") == "Snake Case"

### Formatting Functions

In [ ]:
#| exporti

def big_number(num:float,decimal_places:int=2)->str:
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    formatted = f'%.{decimal_places}f%s' % (num, ['', 'K', 'M', 'B', 'T', 'P'][magnitude])
    return formatted

In [ ]:
#| exporti 

def big_dollars(num:float,decimal_places:int=2)->str:
    formatted = big_number(num,decimal_places)
    return f"${formatted}"

In [ ]:
#| exporti

def as_multiple(num:float)->str:
    if abs(num) < 0.005:
        return '-'
    else:
        return '{:.2f}x'.format(num)

In [ ]:
#| exporti 

def format_minutes(time):
    if type(time)==dt.time:
        return time.strftime("%M:%S")
    minutes = math.floor(time)
    seconds = (time * 60) % 60
    return "%02d:%02d" % (minutes, seconds)

In [ ]:
#| exporti
def format_date(dt):
    try:
        return pd.Timestamp(dt).strftime('%b %d %Y')
    except:
        return dt

In [ ]:
#| exporti
def format_month(dt):
    try:
        return pd.Timestamp(dt).strftime('%b %Y')
    except:
        return dt

In [ ]:
format_month('20220101')

'Jan 2022'

In [ ]:
#| exporti
def as_millions_2dp(num:float)->str:
    return '{:,.2f}mm'.format(num/1e6)

def as_millions(num:float)->str:
    if abs(num) < 0.5e6:
        return '-'
    else:
        return '{:,.0f}mm'.format(num/1e6)

def as_billions(num:float)->str:
    return '%.2fB' % (num/1e9)

def as_thousands_2dp(num:float)->str:
    return '{:,.2f}k'.format(num/1e3)

def as_thousands(num:float)->str:
    return '{:,.0f}k'.format(num/1e3)

In [ ]:
big_number(1e6)

'1.00M'

In [ ]:
assert big_number(1e6) == '1.00M'
assert big_number(1234567)=='1.23M'

assert big_dollars(1e6)=='$1.00M'
assert big_dollars(1234567891) == '$1.23B'

assert as_multiple(1.2) == "1.20x"

assert format_minutes(60.5) == '60:30'

assert format_date('20220212') == 'Feb 12 2022'
assert format_date('') == ''
assert format_month('20220212') == 'Feb 2022'

In [ ]:
assert as_millions(123456799) == '123mm'

In [ ]:
assert as_millions_2dp(123456799) == '123.46mm'
assert as_billions(123456789) == '0.12B'
assert as_thousands(12345) == '12k'

### Formatter

In [ ]:
#| export

class Formatter():
    """
    A customizable object for applying string formats. 
    
    Any additional attributes passed during instantion must have a name ending in "_format". 
    Values can be either a formattable string (e.g. "{:.0f}") or a callable.  
    """
    def __init__(
        self,
        null_format: str = '',
        dollars_format: FormatterT = "${:,.0f}",
        percent_format: FormatterT = "{:.0%}",
        percent2dp_format: FormatterT = "{:.2%}",
        number_format: FormatterT = "{:,.0f}",
        small_number_format: FormatterT = "{:.2f}",
        big_number_format: FormatterT = big_number,
        big_dollars_format: FormatterT = big_dollars,
        multiple_format: FormatterT = as_multiple,
        minutes_format: FormatterT = format_minutes,
        date_format: FormatterT = format_date,
        month_format: FormatterT = format_month,
        millions_format: FormatterT = as_millions,
        millions2dp_format: FormatterT = as_millions_2dp,
        billions_format: FormatterT = as_billions,
        thousands_format: FormatterT = as_thousands,
        thousands2dp_format: FormatterT = as_thousands_2dp,
        **kwargs
    ):
        self.null_format = null_format
        self.dollars_format=dollars_format
        self.percent_format=percent_format
        self.percent2dp_format=percent2dp_format
        self.number_format=number_format
        self.small_number_format=small_number_format
        self.big_number_format=big_number_format
        self.big_dollars_format=big_dollars_format
        self.multiple_format = multiple_format
        self.minutes_format = minutes_format
        self.date_format = date_format
        self.month_format = month_format
        self.millions_format = millions_format
        self.millions2dp_format = millions2dp_format
        self.billions_format = billions_format
        self.thousands_format = thousands_format
        self.thousands2dp_format = thousands2dp_format
        self.no_format = lambda x: x
        self.text_format = lambda x: x
        
        #add anything extra
        for k,v in kwargs.items():
            if not k.split('_')[-1]=='format':
                raise ValueError(f"Keyword arguments passed to the Formatter must end in '_format' ")
            setattr(self,k,v)
        
        # create methods from string_formats
        items = [(k,v) for k,v in self.__dict__.items()]
        for k,v in items:
            if not k.startswith('null'):
                method = '_'.join([x for x in k.split('_')][:-1]) # remove "_format" from the end of the attribute when setting the method
                if type(v)==tuple:
                    setattr(
                        self,
                        method,
                        format_factory(
                            v[0],
                            null_format = self.null_format,
                            **v[1]
                        )
                    )
                else:
                    setattr(
                        self,
                        method,
                        format_factory(v,null_format=self.null_format)
                    )

#### Examples

In [ ]:
fmt = Formatter()



assert fmt.big_dollars(1234567) == big_dollars(1234567) == '$1.23M'
assert fmt.multiple(1.22999)=='1.23x'
assert fmt.percent(.0123)=="1%"
assert fmt.percent2dp(.0123)=="1.23%"
assert fmt.date('20220212')=='Feb 12 2022'
assert fmt.month('20220212')=='Feb 2022'
assert fmt.minutes(60.5)=="60:30"
assert fmt.millions(123456789) == "123mm"
assert fmt.millions2dp(123456789) == "123.46mm"
assert fmt.billions(1234567890) == "1.23B"
assert fmt.no_format('hello') == 'hello'
assert fmt.no_format(2) == 2

#### Formatting Null Values

The `null_format` attribute of the Formatter object is a string that gets returned when any method is called on a null value.  

By default, nulls are represented as empty strings

In [ ]:
import numpy as np

assert fmt.big_dollars(np.nan)==fmt.number(np.nan)==''==fmt.null_format

Formatter will treat empty strings as null.

In [ ]:
assert fmt.percent('')==fmt.null_format

In [ ]:
custom_null_format = "I'm a null value!"
fmt_custom_null = Formatter(null_format=custom_null_format)
assert fmt_custom_null.big_number(np.nan)==fmt_custom_null.number(np.nan)==custom_null_format

Override default implementation at instantiation: 

In [ ]:
fmt_custom_dollars = Formatter(
    dollars_format="${:,.2f}"
)
assert fmt_custom_dollars.dollars(12345.678) == "$12,345.68"

In [ ]:
fmt_custom = Formatter(
    snake_format = convert_snake_case
)
fmt_custom.snake("snake_case")

'snake case'

In [ ]:
# if you need to pass arguments to the function factory 
fmt_custom_with_kwargs = Formatter(
    snake_to_camel_format = (
        convert_snake_case,{'title':True}
    )
)
assert fmt_custom_with_kwargs.snake_to_camel("snake_case")=='Snake Case'

## Colors

In [ ]:
#| export

class ListOfColors(list):
    """
    List of validated colors that can be converted to hex or rgb
    """
    colors:List[Color]
    
    @classmethod
    def __get_validators__(cls):
        # one or more validators may be yielded which will be called in the
        # order to validate the input, each validator will receive as an input
        # the value returned from the previous validator
        yield cls.validate

    @classmethod
    def __modify_schema__(cls, field_schema):
        # __modify_schema__ should mutate the dict it receives in place,
        # the returned value will be ignored
        field_schema.update(
            
            description='List of Colors'
            
        )

    @classmethod
    def validate(cls, v):
        
        return cls(colors=v)
    
    def __init__(self,colors):
        
        self.colors = [Color(x) for x in colors]
        super().__init__(self.colors)
        
    def as_hex(self):
        return [x.as_hex() for x in self.colors]
    def as_rgb(self):
        return [x.as_rgb() for x in self.colors]

In [ ]:
red_green_blue = ListOfColors(['red','green','blue'])
red_green_blue

[Color('red', rgb=(255, 0, 0)),
 Color('green', rgb=(0, 128, 0)),
 Color('blue', rgb=(0, 0, 255))]

In [ ]:
#| export 

class PretiumColors(BaseModel):
    color_format: Literal['hex','rgb'] = 'hex'
    pretium_blue: Color = "#1f0ee2"
    progress_green: Color = 'teal'
    real_estate: Color = '#FF9900'
    resi_credit: Color = "rgb(118,78,159)"
    credit: Color = 'rgb(225,225,179)'
    holdings: Color = 'rgb(128,177,211)'
    sequential: ListOfColors = [
        '#1F0EE2',
        '#A3CBD6',
        '#DAEAEF',
        '#70878B',
        '#BCC2C5',
        '#3A7382',
        '#62A6B9',
        '#375570',
        '#759ABB',
        '#D1DDE8',
        '#9EAEB1',
        '#DEE1E2'
    ]
    
    @root_validator
    def _validate_colors(cls,values):
        for k,v in values.items():
            if not isinstance(v,Color) and k!='color_format':
                try:
                    if isinstance(v,list):
                        values[k]=ListOfColors(v)
                    else:
                        values[k]=Color(v)
                except Exception as e:
                    raise ValueError(f"'{k}'='{v}' couldn't be converted to a Color. {e}")
        return values
    
    @root_validator(skip_on_failure=True)
    def _convert_to_color_format(cls,values):
        color_format = values.pop('color_format')
        if color_format == 'hex':
            values = {k:v.as_hex() for k,v in values.items()}
        elif color_format == 'rgb':
            values = {k:v.as_rgb() for k,v in values.items()}
        values['color_format'] = color_format
        return values
    
    class Config:
        extra = 'allow'

In [ ]:
colors = PretiumColors()
# all hex by default
colors.dict()

{'pretium_blue': '#1f0ee2',
 'progress_green': '#008080',
 'real_estate': '#f90',
 'resi_credit': '#764e9f',
 'credit': '#e1e1b3',
 'holdings': '#80b1d3',
 'sequential': ['#1f0ee2',
  '#a3cbd6',
  '#daeaef',
  '#70878b',
  '#bcc2c5',
  '#3a7382',
  '#62a6b9',
  '#375570',
  '#759abb',
  '#d1dde8',
  '#9eaeb1',
  '#dee1e2'],
 'color_format': 'hex'}

In [ ]:
rgb_colors = PretiumColors(color_format='rgb')
rgb_colors.dict()

{'pretium_blue': 'rgb(31, 14, 226)',
 'progress_green': 'rgb(0, 128, 128)',
 'real_estate': 'rgb(255, 153, 0)',
 'resi_credit': 'rgb(118, 78, 159)',
 'credit': 'rgb(225, 225, 179)',
 'holdings': 'rgb(128, 177, 211)',
 'sequential': ['rgb(31, 14, 226)',
  'rgb(163, 203, 214)',
  'rgb(218, 234, 239)',
  'rgb(112, 135, 139)',
  'rgb(188, 194, 197)',
  'rgb(58, 115, 130)',
  'rgb(98, 166, 185)',
  'rgb(55, 85, 112)',
  'rgb(117, 154, 187)',
  'rgb(209, 221, 232)',
  'rgb(158, 174, 177)',
  'rgb(222, 225, 226)'],
 'color_format': 'rgb'}

In [ ]:
PretiumColors(BlueGreenRed = ['blue','green','red']).dict()

{'pretium_blue': '#1f0ee2',
 'progress_green': '#008080',
 'real_estate': '#f90',
 'resi_credit': '#764e9f',
 'credit': '#e1e1b3',
 'holdings': '#80b1d3',
 'sequential': ['#1f0ee2',
  '#a3cbd6',
  '#daeaef',
  '#70878b',
  '#bcc2c5',
  '#3a7382',
  '#62a6b9',
  '#375570',
  '#759abb',
  '#d1dde8',
  '#9eaeb1',
  '#dee1e2'],
 'BlueGreenRed': ['#00f', '#008000', '#f00'],
 'color_format': 'hex'}

In [ ]:
# adding custom colors
colors = PretiumColors(
    other_color='darkblue',
    
)
colors.other_color

'#00008b'

In [ ]:
from pydantic import ValidationError

try:
    PretiumColors(bad='<not a proper color>')
except ValidationError as e:
    print(e)

1 validation error for PretiumColors
__root__
  'bad'='<not a proper color>' couldn't be converted to a Color. value is not a valid color: string not recognised as a valid color (type=value_error)


## Plotly Templates

In [ ]:
#| exporti 

pio.templates['top-legend'] = go.layout.Template(
    layout_paper_bgcolor='rgb(248, 248, 255)',
    layout_plot_bgcolor='rgb(248, 248, 255)',
    layout_margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    layout_legend=dict(
        orientation = 'h',
        yanchor='top',
        y=1.10,
        xanchor='left',
        x=0
    )
)

In [ ]:
#| include: false
!nbdev_build_lib

Converted 00_core.ipynb.
Converted 01_credit_data.ipynb.
Converted 01_data.ipynb.
Converted 01_kpi_data.ipynb.
Converted 01_models.data.ipynb.
Converted 01_resicredit_data.ipynb.
Converted 02_reportlab_document.ipynb.
Converted 77_formatting.ipynb.
Converted 78_plotting.ipynb.
Converted 88_temp_static.ipynb.
Converted 99_utilities.ipynb.
Converted anchor_reporting.ipynb.
Converted index.ipynb.
Converted 10_models.reportlab.ipynb.
Converted 11_models.report_definitions.ipynb.
Converted 12_models.create_document.ipynb.
Converted selene_kpi_queries.ipynb.
Converted KPI Report Master.ipynb.
Converted anchor_risk.ipynb.
Converted credit_risk.ipynb.
Converted kpi_report.ipynb.
Converted kpi_report_support_funcs.ipynb.
Converted reportlab_test.ipynb.
Converted resicredit_risk.ipynb.
Converted sfr_risk.ipynb.
Converted anchor_calcs.ipynb.
Converted credit_calcs.ipynb.
Converted dephaven_kpis.ipynb.
Converted p3_redemption_risk.ipynb.
Converted progress_kpis.ipynb.
Converted resi_credit.ipynb.
